In [7]:
from collections import Counter

import torch
import torchaudio
import numpy as np

import muspy
import pypianoroll as pypir

from utils import pianoroll2binaryroll, binaryroll2pitchlist, pitchlist2pianoroll, shortpitchlist2pitchlist, midi2audio, add_padding_bar
from utils import UNIT_AS_TICK, BEAT_AS_TICK, BAR_AS_TICK

PATH_PREFIX = 'data_set/'
PATH_PREFIX_Q = 'quantized/'

In [8]:
relative_semitones_major = {
    'F#': -6,
    'Gb': -6,
    'G': -5,
    'G#': -4,
    'Ab': -4,
    'A': -3,
    'A#': -2,
    'Bb': -2,
    'B': -1,
    'C': 0,
    'C#': 1,
    'Db': 1,
    'D': 2,
    'D#': 3,
    'Eb': 3,
    'E': 4,
    'F': 5
}

relative_semitones_minor = {
    'D#': -6,
    'Eb': -6,
    'E': 5,
    'F': 4,
    'F#': -3,
    'Gb': -3,
    'G': -2,
    'G#': -1,
    'Ab': -1,
    'A': 0,
    'A#': 1,
    'Bb': 1,
    'B': 2,
    'C': 3,
    'C#': 4,
    'Db': 4,
    'D': 5,
}

In [9]:
from pathlib import Path

data_dir = Path(PATH_PREFIX)
file_list = sorted(list(data_dir.rglob('*.mid')))

file_list = [file.name for file in file_list]

In [15]:
for filename in file_list:
    midi_data = muspy.read(f'{PATH_PREFIX}{filename}')

    # song_attrs = { 0: song title, 1: time sig, 2: key sig, 3: bpm, 4: pair number, 5: num bar, 6: call / res }
    song_attrs = filename.split('.')[0].split('-')
    bpm = float(song_attrs[3])
    key_and_mode = song_attrs[2]

    tempo_cleaned = midi_data.deepcopy()
    new_tempos = [muspy.Tempo(0, bpm)]

    tempo_cleaned.tempos = new_tempos

    transposed = tempo_cleaned.deepcopy()
    semitone_dict = relative_semitones_minor if 'm' in key_and_mode else relative_semitones_major
    
    key = key_and_mode[:-1] if len(key_and_mode) > 1 else key_and_mode
    
    transposed.transpose(-1 * semitone_dict[key])
        
    pianoroll_obj = muspy.to_pypianoroll(transposed)
    pianoroll = pianoroll_obj.tracks[0].pianoroll
    
    quantized_pianoroll_obj = pianoroll_obj.copy()
    
    before_quantized_pianoroll = pianoroll.copy()
    before_quantized_pianoroll_binaryroll = pianoroll2binaryroll(before_quantized_pianoroll)
    before_quantized_pianoroll_pitch_list = binaryroll2pitchlist(before_quantized_pianoroll_binaryroll)
    
    len_pitch_list = len(before_quantized_pianoroll_pitch_list)

    short_quantized_pianoroll_pitch_list = []

    for i in range(0, len_pitch_list, UNIT_AS_TICK):
        sliced = before_quantized_pianoroll_pitch_list[i : i + UNIT_AS_TICK]

        pitch_cnt_dict = dict(Counter(sliced))
        
        max_pitch_cnt_value = max(pitch_cnt_dict.values())
        max_pitch_cnt_pitch = [key for key, val in pitch_cnt_dict.items() if val == max_pitch_cnt_value][0]
        
        short_quantized_pianoroll_pitch_list.append(max_pitch_cnt_pitch)
        
    quantized_pianoroll_pitch_list = shortpitchlist2pitchlist(short_quantized_pianoroll_pitch_list)
    quantized_pianoroll_obj.tracks[0].pianoroll = pitchlist2pianoroll(quantized_pianoroll_pitch_list)
    
    quantized_song = muspy.from_pypianoroll(quantized_pianoroll_obj)

    # song_attrs = { 0: song title, 1: time sig, 2: key sig, 3: bpm, 4: pair number, 5: num bar, 6: call / res }
    quantized_filename = f'{song_attrs[0]}-{song_attrs[1]}-{song_attrs[3]}-{song_attrs[4]}-{song_attrs[5]}-{song_attrs[6]}.mid'
    quantized_file_path = f'{PATH_PREFIX_Q}{quantized_filename}'
    
    muspy.write_midi(quantized_file_path, quantized_song)